In [1]:
import pandas as pd
import pyarrow
import gc
from pathlib import Path

In [2]:
df = pd.read_parquet(
        'dataset/data.parquet',
        engine="pyarrow"
    )
df.head()

,pravogovruNd,issuedByIPS,docdateIPS,docNumberIPS,doc_typeIPS,headingIPS,doc_author_normal_formIPS,signedIPS,statusIPS,actual_datetimeIPS,actual_datetime_humanIPS,is_widely_used,textIPS,classifierByIPS,keywordsByIPS,text_clean
0,102685017,Постановление Правительства Российской Федерации,28.02.2020,210,Постановление,О внесении изменений в Правила предоставления ...,Правительство Российской Федерации,Мишустин М.,Действует без изменений,1710728935.8167202,Mon Mar 18 05:28:55 2024,0,\nПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ\n \nПОСТ...,"010.140.030.010.000$Отмена, изменение и дополн...","АКЦИОНЕРНЫЕ, БЮДЖЕТ, ВЗНОС, ВНЕШНЕЭКОНОМИЧЕСКА...",правительство российской федерации постановлен...
1,605953582,Приказ Федеральной таможенной службы,23.08.2023,751,Приказ,О внесении изменений в Порядок проведения тамо...,Федеральная таможенная служба,nan,Действует без изменений,1710772322.8312817,Mon Mar 18 17:32:02 2024,0,\nФЕДЕРАЛЬНАЯ ТАМОЖЕННАЯ СЛУЖБА \n \nПРИКАЗ\n...,UNKNOWN,,федеральная таможенная служба приказ москва а...
2,102386309,Приказ Федеральной антимонопольной службы,22.12.2015,1347/15,Приказ,"Об утверждении коэффициентов сезонности, приме...",Федеральная антимонопольная служба,Артемьев И.,Действует без изменений,1710441798.790414,Thu Mar 14 21:43:18 2024,1,ФЕДЕРАЛЬНАЯ АНТИМОНОПОЛЬНАЯ СЛУЖБА \n ПРИКАЗ ...,UNKNOWN,,федеральная антимонопольная служба приказ моск...
3,102100389,Распоряжение Правительства Российской Федерации,12.10.2005,1654-р,Распоряжение,О передаче в государственную собственность Омс...,Правительство Российской Федерации,Фрадков М.,Действует без изменений,1710335188.5398128,Wed Mar 13 16:06:28 2024,0,\nПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ\n \nРАСП...,010.070.020.040.290$Омская область $030.030.01...,"ВОЕННЫЙ, ОБЛАСТЬ, ПАМЯТНИК, ПЕРЕДАЧА, СОБСТВЕН...",правительство российской федерации распоряжени...
4,102466500,Постановление Правительства Российской Федерации,30.03.2018,352,Постановление,О внесении изменений в государственную програм...,Правительство Российской Федерации,Медведев Д.,Утратил силу,1710375877.834703,Thu Mar 14 03:24:37 2024,0,\nПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ\n \nПОСТ...,"010.140.030.010.000$Отмена, изменение и дополн...","ГОСУДАРСТВЕННЫЙ, ИЗМЕНЕНИЕ, ПРОГРАММА, СРЕДА, ...",правительство российской федерации постановлен...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49727 entries, 0 to 49999
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   pravogovruNd               49727 non-null  object
 1   issuedByIPS                49727 non-null  object
 2   docdateIPS                 49727 non-null  object
 3   docNumberIPS               49727 non-null  object
 4   doc_typeIPS                49727 non-null  object
 5   headingIPS                 49727 non-null  object
 6   doc_author_normal_formIPS  49727 non-null  object
 7   signedIPS                  49727 non-null  object
 8   statusIPS                  49727 non-null  object
 9   actual_datetimeIPS         49727 non-null  object
 10  actual_datetime_humanIPS   49727 non-null  object
 11  is_widely_used             49727 non-null  object
 12  textIPS                    49727 non-null  object
 13  classifierByIPS            49727 non-null  object
 14  keywordsByI

In [2]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer

nltk.download('stopwords')
# nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /home/hedg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/hedg/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [5]:
# remove or replace empty
df = df.dropna(subset=['textIPS'])

df['classifierByIPS'] = df['classifierByIPS'].fillna('UNKNOWN')
df['keywordsByIPS'] = df['keywordsByIPS'].fillna('')

df = df.drop(columns=['taggedtextIPS'])

In [6]:
def clean(text):
    text = text.lower()                                 
    text = re.sub(r'\s+', ' ', text)                    
    text = re.sub(r'\d+', '', text)                     
    text = re.sub(r'[^\w\s]', '', text) 
    return text.strip()

In [7]:
# clean data (to lowercase, remove punctuation, numbers and many spaces)
df['text_clean'] = df['textIPS'].apply(clean)
df.to_parquet('dataset/data.parquet')

In [ ]:
# tokenize and remove stopwords
russian_stopwords = set(stopwords.words("russian"))

def tokenize_and_filter(text):
    tokens = word_tokenize(text, language='russian')
    return [word for word in tokens if word not in russian_stopwords]

# df['tokens'] = df['text_clean'].apply(tokenize_and_filter)

In [5]:
gc.collect()

0

In [ ]:
chunk_size = 3000
total = len(df)

for i in range(0, total, chunk_size):
    print(f"Processing chunk {i // chunk_size + 1} of {total // chunk_size + 1}")
    chunk = df.iloc[i:i+chunk_size].copy()
    
    # tokenize
    chunk['tokens'] = chunk['text_clean'].apply(tokenize_and_filter)
    
    # save to disk
    chunk_path = f"ruslowod_tokens_chunk_{i//chunk_size}.parquet"
    chunk.to_parquet(chunk_path)
    
    # free memory
    del chunk
    gc.collect()

Processing chunk 1 of 17
Processing chunk 2 of 17
Processing chunk 3 of 17
Processing chunk 4 of 17
Processing chunk 5 of 17
Processing chunk 6 of 17
Processing chunk 7 of 17
Processing chunk 8 of 17
Processing chunk 9 of 17
Processing chunk 10 of 17
Processing chunk 11 of 17
Processing chunk 12 of 17
Processing chunk 13 of 17
Processing chunk 14 of 17
Processing chunk 15 of 17
Processing chunk 16 of 17
Processing chunk 17 of 17


In [4]:
# Lemmatization
morph = MorphAnalyzer()

def lemmatize(tokens):
    return [morph.parse(token)[0].normal_form for token in tokens]

In [2]:
data_dir = Path(".")

# get all token chunk files
chunk_files = sorted(data_dir.glob("ruslowod_tokens_chunk_*.parquet"))

In [ ]:
for i, chunk_file in enumerate(chunk_files):
    print(f"Lemmatizing chunk {i+1}/{len(chunk_files)}: {chunk_file.name}")
    
    chunk = pd.read_parquet(chunk_file)

    # lemmatize
    chunk['lemmas'] = chunk['tokens'].apply(lemmatize)
    chunk['lemmatized_text'] = chunk['lemmas'].apply(lambda x: ' '.join(x))

    del chunk['lemmas']

    output_file = chunk_file.with_name(chunk_file.name.replace("tokens", "lemmatized"))
    chunk.to_parquet(output_file, index=False)

    # cleanup memory
    del chunk
    gc.collect()

Lemmatizing chunk 1/17: ruslowod_tokens_chunk_0.parquet
Lemmatizing chunk 2/17: ruslowod_tokens_chunk_1.parquet
Lemmatizing chunk 3/17: ruslowod_tokens_chunk_10.parquet
Lemmatizing chunk 4/17: ruslowod_tokens_chunk_11.parquet
Lemmatizing chunk 5/17: ruslowod_tokens_chunk_12.parquet
Lemmatizing chunk 6/17: ruslowod_tokens_chunk_13.parquet
Lemmatizing chunk 7/17: ruslowod_tokens_chunk_14.parquet
Lemmatizing chunk 8/17: ruslowod_tokens_chunk_15.parquet
Lemmatizing chunk 9/17: ruslowod_tokens_chunk_16.parquet
Lemmatizing chunk 10/17: ruslowod_tokens_chunk_2.parquet


KeyboardInterrupt: 

In [5]:

for i, chunk_file in enumerate(chunk_files[9:], 9):
    print(f"Lemmatizing chunk {i+1}/{len(chunk_files)}: {chunk_file.name}")
    
    chunk = pd.read_parquet(chunk_file)

    # lemmatize
    chunk['lemmas'] = chunk['tokens'].apply(lemmatize)
    chunk['lemmatized_text'] = chunk['lemmas'].apply(lambda x: ' '.join(x))

    del chunk['lemmas']

    output_file = chunk_file.with_name(chunk_file.name.replace("tokens", "lemmatized"))
    chunk.to_parquet(output_file, index=False)

    # cleanup memory
    del chunk
    gc.collect()

Lemmatizing chunk 10/17: ruslowod_tokens_chunk_2.parquet


NameError: name 'lemmatize' is not defined

In [3]:
# get all ready chunks files
PATTERN = "ruslowod_lemmatized_chunk_*.parquet" 
files = list(data_dir.glob(PATTERN))

In [ ]:
# concat by 4 files and save each as preprocessed
for i in range(0, len(files), 4):
    dfs = []
    for f in files[i:i+4]:
        df = pd.read_parquet(
            f,
            engine="pyarrow"
        )
        dfs.append(df)
    df_full = pd.concat(dfs, ignore_index=True)
    df_full.to_parquet(f'dataset/preproc_data{i//4}.parquet')
    del dfs
    gc.collect()